In [39]:
import configparser
import itertools
import os
from typing import Dict, List
from langchain_core.tools import tool

@tool
def generate_ini_files(ini_input: str, param_study: Dict[str, str]):
    """
    Generate multiple INI files for a parameter study based on the 
    input template and parameter range.

    Args:
        ini_input (str): The base INI file content as a string.
        param_study (Dict[str, str]): A dictionary where keys are INI keys and values are parameter ranges in the format "start,end,stepsize".

    """
    # Parse the input INI string
    config = configparser.ConfigParser()
    config.read_string(ini_input)
    output_dir = "parameter_study_output"

    # Prepare parameter ranges from param_study
    param_ranges = {}
    for key, range_str in param_study.items():
        start, end, step = map(float, range_str.split(","))
        param_ranges[key] = [round(x, 6) for x in frange(start, end, step)]

    # Generate combinations of parameters
    # Friendly reminder '*' is the unpacking operator, i.e.:
    # [('a', [1,2]), ('b', [3,4])] -> unpacked by * -> ('a', [1,2]), ('b', [3,4])
    param_keys, param_values = zip(*param_ranges.items()) if param_ranges else ([], [])
    param_combinations = list(itertools.product(*param_values))

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Generate INI files
    for idx, combination in enumerate(param_combinations):
        # Update the config with the current combination of parameters
        for key, value in zip(param_keys, combination):
            section, option = key.split("/", 1)
            config[section][option] = str(value)

        # Write the updated config to a new INI file
        output_path = os.path.join(output_dir, f"study_case_{idx + 1}.ini")
        with open(output_path, "w") as f:
            config.write(f)

    return f"Generated {len(param_combinations)} INI files in '{output_dir}'."

def frange(start: float, end: float, step: float) -> List[float]:
    """
    Generate a range of floating-point numbers.

    Args:
        start (float): Start value.
        end (float): End value.
        step (float): Step size.

    Returns:
        List[float]: List of floating-point numbers.
    """
    while start <= end:
        yield start
        start += step

# Example usage
ini_template = """[E3DGeometryData]

[E3DGeometryData/Machine]
Type=SingleScrew
Unit=mm
Zwickel=0
MachineName=TestMachine
RotationDirection=CW
BarrelDiameter=25
CenterlineDistance=12.5
BarrelStraightCut=5
NoOfElements=10
NoOfFlights=2
BarrelLength=500

[E3DProcessParameters]
ScrewSpeed=100
ProcessType=Extrusion
MassThroughput=5
MaterialTemperature=200
BarrelTemperature=180
ScrewTemperature=170
BarrelTemperatureAdiabatic=False
ScrewTemperatureAdiabatic=False
"""

param_study = {
    "E3DProcessParameters/ScrewSpeed": "50,150,25",
    "E3DProcessParameters/MaterialTemperature": "190,210,10"
}

#generate_ini_files(ini_template, param_study)


In [10]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.model_name
llm.temperature

0.7

In [40]:
from langchain_core.tools import tool 

tools = [generate_ini_files]

In [41]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

prompt_v2_str = """ 
Hallo
param_study = {{
    "E3DProcessParameters/ScrewSpeed": "50,150,25",
}}

"""

prompt_v1_str = """ 
You are a helpful assistant for creating parameter studies based on INI files. A user provides you 
with an INI file as text and a natural language description of the parameter study. Here is an example:

This could be the INI file as text:
[E3DGeometryData]
[E3DGeometryData/Machine]
Type=SingleScrew
Unit=mm
Zwickel=0
MachineName=TestMachine
RotationDirection=CW
BarrelDiameter=25
CenterlineDistance=12.5
BarrelStraightCut=5
NoOfElements=10
NoOfFlights=2
BarrelLength=500
[E3DProcessParameters]
ScrewSpeed=100
ProcessType=Extrusion
MassThroughput=5
MaterialTemperature=200
BarrelTemperature=180
ScrewTemperature=170
BarrelTemperatureAdiabatic=False
ScrewTemperatureAdiabatic=False

The natural language description of the parameter study could be:
Example1:
'Make a parameter study for the E3DProcessParameters/ScrewSpeed parameter from start value 50 to a max value of 150 
in increments of 25.'
In this case you would call the tool with arguments:
ini_template = the ini file provided by the user
param_study = {{
    "E3DProcessParameters/ScrewSpeed": "50,150,25",
}}
Example2:
'Make a parameter study for the E3DProcessParameters/MaterialTemperature parameter from 190 to 210 in steps of 10.' 
In this case you would call the tool with arguments:
ini_template = the ini file provided by the user
param_study = {{
    "E3DProcessParameters/MaterialTemperature": "190,210,10"
}}

Example3:
'Make a study for the E3DProcessParameters/ScrewSpeed parameter from 50 to 150 in steps of 25 and for the 
E3DProcessParameters/MaterialTemperature parameter from 190 to 210 in steps of 10'
In this case you would call the tool with arguments:
ini_template = the ini file provided by the user
param_study = {{
    "E3DProcessParameters/ScrewSpeed": "50,150,25",
    "E3DProcessParameters/MaterialTemperature": "190,210,10"
}}
"""

prompt_v1 = ChatPromptTemplate([
    ("system", prompt_v1_str),
    ("user", "{request}")
])

In [42]:
prompt_v1.invoke({"request": "Make a parameter study for the E3DProcessParameters/ScrewSpeed parameter from start value 50 to a max value of 150 in increments of 25."})

ChatPromptValue(messages=[SystemMessage(content=' \nYou are a helpful assistant for creating parameter studies based on INI files. A user provides you \nwith an INI file as text and a natural language description of the parameter study. Here is an example:\n\nThis could be the INI file as text:\n[E3DGeometryData]\n[E3DGeometryData/Machine]\nType=SingleScrew\nUnit=mm\nZwickel=0\nMachineName=TestMachine\nRotationDirection=CW\nBarrelDiameter=25\nCenterlineDistance=12.5\nBarrelStraightCut=5\nNoOfElements=10\nNoOfFlights=2\nBarrelLength=500\n[E3DProcessParameters]\nScrewSpeed=100\nProcessType=Extrusion\nMassThroughput=5\nMaterialTemperature=200\nBarrelTemperature=180\nScrewTemperature=170\nBarrelTemperatureAdiabatic=False\nScrewTemperatureAdiabatic=False\n\nThe natural language description of the parameter study could be:\nExample1:\n\'Make a parameter study for the E3DProcessParameters/ScrewSpeed parameter from start value 50 to a max value of 150 \nin increments of 25.\'\nIn this case you

In [43]:
llm_with_tools = llm.bind_tools(tools)
chain = prompt_v1 | llm_with_tools

In [44]:
request_str = """ 
Here is an INI file as text:
[E3DGeometryData]
[E3DGeometryData/Machine]
Type=SingleScrew
Unit=mm
Zwickel=0
MachineName=TestMachine
RotationDirection=CW
BarrelDiameter=25
CenterlineDistance=12.5
BarrelStraightCut=5
NoOfElements=1
NoOfFlights=1
BarrelLength=50
[E3DProcessParameters]
ScrewSpeed=100
ProcessType=Extrusion
MassThroughput=5
MaterialTemperature=200
BarrelTemperature=180
ScrewTemperature=170
BarrelTemperatureAdiabatic=False
ScrewTemperatureAdiabatic=False
Make a parameter study for the E3DProcessParameters/ScrewSpeed parameter from start value 50 to a max value of 150 in increments of 25.
"""

In [45]:
reply = chain.invoke({"request": request_str})

In [46]:
reply.tool_calls

[{'name': 'generate_ini_files',
  'args': {'ini_input': '[E3DGeometryData]\n[E3DGeometryData/Machine]\nType=SingleScrew\nUnit=mm\nZwickel=0\nMachineName=TestMachine\nRotationDirection=CW\nBarrelDiameter=25\nCenterlineDistance=12.5\nBarrelStraightCut=5\nNoOfElements=1\nNoOfFlights=1\nBarrelLength=50\n[E3DProcessParameters]\nScrewSpeed=100\nProcessType=Extrusion\nMassThroughput=5\nMaterialTemperature=200\nBarrelTemperature=180\nScrewTemperature=170\nBarrelTemperatureAdiabatic=False\nScrewTemperatureAdiabatic=False',
   'param_study': {'E3DProcessParameters/ScrewSpeed': '50,150,25'}},
  'id': 'call_izul17P3RoX4HcMNZcGY1YA5',
  'type': 'tool_call'}]

In [47]:
tool_mapping = {
    "generate_ini_files": generate_ini_files,
}

In [48]:
from langchain_core.messages import HumanMessage, ToolMessage
messages = []
for tool_call in reply.tool_calls:
    tool = tool_mapping[tool_call['name'].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [49]:
messages

[ToolMessage(content="Generated 5 INI files in 'parameter_study_output'.", tool_call_id='call_izul17P3RoX4HcMNZcGY1YA5')]